In [1]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
import xgboost
from lightgbm import LGBMClassifier
from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input, decode_predictions
from keras.applications.resnet50 import ResNet50
import seaborn
import matplotlib.pyplot as plt

from data_utils import *
from modeling import *

% matplotlib inline
% load_ext autoreload
% autoreload 2


Using TensorFlow backend.


 multilabel_classes: Dictionary, mapping filepaths to numpy array of (multiple) classes 
        (e.g.  {'folder/some_image.jpg': np.array([ 0.,  1.,  0.,  1.]),...})

In [3]:
# TODO: These really should be env variables
PLANET_KAGGLE_ROOT = '/mnt/data/planet'
PLANET_KAGGLE_LABEL_CSV = os.path.join(PLANET_KAGGLE_ROOT, 'train_v2.csv')
PLANET_KAGGLE_JPEG_DIR = os.path.join(PLANET_KAGGLE_ROOT, 'train-jpg')
PLANET_KAGGLE_JPEG_VAL_DIR = os.path.join(PLANET_KAGGLE_ROOT, 'validate-jpg')
assert os.path.exists(PLANET_KAGGLE_ROOT)
assert os.path.exists(PLANET_KAGGLE_JPEG_DIR)
assert os.path.exists(PLANET_KAGGLE_LABEL_CSV)
assert os.path.exists(PLANET_KAGGLE_JPEG_VAL_DIR)

In [4]:
labels_df = pd.read_csv(PLANET_KAGGLE_LABEL_CSV)
labels_df.head()

,image_name,tags
0,train_0,haze primary
1,train_1,agriculture clear primary water
2,train_2,clear primary
3,train_3,clear primary
4,train_4,agriculture clear habitation primary road


In [5]:
label_list = labels_from(labels_df)

In [6]:
labels_df = enrich_with_feature_encoding(labels_df)

In [7]:
multi_label_dict = to_multi_label_dict(labels_df)

In [8]:
nb_train_samples = get_file_count(os.path.join(PLANET_KAGGLE_JPEG_DIR, '*.jpg'))
nb_validation_samples = get_file_count(os.path.join(PLANET_KAGGLE_JPEG_VAL_DIR, '*.jpg'))

In [9]:
model = ResNet50(include_top=False)

In [10]:
train_features = []
for i in tqdm(range(35000)):
    img_path = os.path.join(PLANET_KAGGLE_JPEG_DIR, 'train_{}.jpg'.format(i))
    img = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    train_features.append(model.predict(x).ravel())

100%|██████████| 35000/35000 [23:57<00:00, 24.35it/s]


In [11]:
train_features = np.array(train_features)

In [12]:
validation_features = []
for i in tqdm(range(35000, 40479)):
    img_path = os.path.join(PLANET_KAGGLE_JPEG_VAL_DIR, 'train_{}.jpg'.format(i))
    img = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    validation_features.append(model.predict(x).ravel())

100%|██████████| 5479/5479 [03:43<00:00, 24.47it/s]


In [13]:
validation_features = np.array(validation_features)

In [29]:
random_seed=42

# Prepare data
y_train = np.array([multi_label_dict['train_{}'.format(i)] for i in range(35000)])
y_val = np.array([multi_label_dict['train_{}'.format(i)] for i in range(35000, 40479)])

In [34]:
xgb_model = xgboost.XGBClassifier(max_depth=7, learning_rate=0.1, n_estimators=300, \
                                  silent=True, objective='binary:logistic', nthread=-1, \
                                  gamma=0, min_child_weight=1, max_delta_step=0, \
                                  subsample=1, colsample_bytree=1, colsample_bylevel=1, \
                                  reg_alpha=0, reg_lambda=1, scale_pos_weight=1, \
                                  base_score=0.5, seed=random_seed, missing=None)

In [35]:
def train_and_validate(model, train_features, train_labels, validation_features):
    n_classes = train_labels.shape[1]
    y_val_pred = np.zeros((validation_features.shape[0], n_classes))
    for class_i in tqdm(range(n_classes)): 
        model.fit(train_features, train_labels[:, class_i])
        y_val_pred[:, class_i] = model.predict_proba(validation_features)[:, 1]
    return y_val_pred

In [32]:
results_dict = dict()

In [36]:
with Timer() as t:
    y_val_pred = train_and_validate(xgb_model, train_features, y_train, validation_features)

100%|██████████| 17/17 [36:34<00:00, 109.04s/it]


In [37]:
results_dict['xgb']={
    'time': t.interval,
    'f2': f2_pred(y_val, threshold_prediction(y_val_pred, threshold=0.1))
}

In [38]:
lgbm_model = LGBMClassifier(num_leaves=(2**7)-1, learning_rate=0.1, n_estimators=300, silent=True, seed=random_seed)

In [39]:
with Timer() as t:
    y_val_pred = train_and_validate(lgbm_model, train_features, y_train, validation_features)

100%|██████████| 17/17 [11:14<00:00, 26.75s/it]


In [40]:
results_dict['lgbm']={
    'time': t.interval,
    'f2': f2_pred(y_val, threshold_prediction(y_val_pred, threshold=0.1))
}

In [41]:
print(results_dict)

{'lgbm': {'f2': 0.91587689477618017, 'time': '664.159'}, 'xgb': {'f2': 0.91455112015089102, 'time': '2159.710'}}
